# Purpose of this notebook

The main interesting KOOP SRU repositories may be BWB and CVDR, an PUC/POD while it works,\
which have their own notebooks here exploring them.

There are other repositories that we can access and/or are in the documention, let's at least mention them here.

In [1]:
import collections, pprint, json, collections

import wetsuite.helpers.etree
import wetsuite.helpers.localdata
import wetsuite.helpers.koop_parse
import wetsuite.helpers.format
import wetsuite.helpers.notebook

In [21]:
frbr_fetched  = wetsuite.helpers.localdata.LocalKV('frbr_fetched.db', key_type=str, value_type=bytes )

# PLOOI

PLOOI is in a redesign phase, so is currently not working.

In [2]:
unktypes = collections.defaultdict(int)

def plooi_callback(record):
    #recordSchema   = record.find('recordSchema')      # e.g. <recordSchema>http://standaarden.overheid.nl/sru/</recordSchema>
    #recordPacking  = record.find('recordPacking')     # probably <recordPacking>xml</recordPacking>
    recordData     = record.find('recordData')        # the actual record 
    recordPosition = record.find('recordPosition')    # e.g. <recordPosition>12</recordPosition>
    print( '====== record %s ====='%recordPosition.text )
    payload = recordData[0] # gzd
    
    wetsuite.helpers.etree.indent_inplace(payload)
    print( wetsuite.helpers.etree.tostring( payload, encoding='unicode' ) )

    # TODO: figure out data model. 
    # Currently we flatten everything into one 
    
    ###
    originalData = payload.find('originalData')
    owmskern     = wetsuite.helpers.etree.kvelements_to_dict( originalData.find('meta/owmskern')   )
    owmsmantel   = wetsuite.helpers.etree.kvelements_to_dict( originalData.find('meta/owmsmantel') )
    bronmeta     = wetsuite.helpers.etree.kvelements_to_dict( originalData.find('meta/othermeta/bronmeta') )
    plooiipm     = wetsuite.helpers.etree.kvelements_to_dict( originalData.find('meta/plooiipm'), ignore_tagnames=('extrametadata',) )
    for node in originalData.find('meta/plooiipm'):
        if node.tag == 'extrametadata':
            plooiipm[ node.get('name') ] = node.text

    #TODO: parse other parts
    
    merged = {}
    merged.update(owmskern)
    merged.update(owmsmantel)
    merged.update(bronmeta)
    merged.update(plooiipm)
    #pprint.pprint(merged)
            
    ### 
    enrichedData = payload.find('enrichedData')
    for pu in enrichedData.findall('publicatieurl'):
        typ = pu.get('type') 
        if typ in ('xml', 
                    #'html',
                    'pdf',
                    ): 
            try:
                url = pu.text
                url = url.replace('http://beta-zoekservice1.overheid.nl/repository-plooi/', 'https://open.overheid.nl/repository/')
                print( url)
                #db.cached_fetch( url, verbose=True, backoff_sec=0.1, given_conn=conn )
            except Exception as e:
                print( e )
                #raise
        
        #unktypes[e.get('manifestation')] += 1


from wetsuite.datacollect.koop_repositories import PLOOI
sru_plooi = PLOOI( verbose=True )

#print( sru_plooi._url() )

if 1:
    pprint.pprint( sru_plooi.explain() )
    #pprint.pprint( sru_plooi.explain_parsed() )


if 0:
    #sru_plooi.search_retrieve_many('plooi.topthema = "ruimte en infrastructuur"', up_to=50000, at_a_time=100, callback=plooi_callback)

    #sru_plooi.search_retrieve('dcterms.type = "beslissing op verzoek art. 3 Wob"')
    #print( sru_plooi.num_records() )
    sru_plooi.search_retrieve_many('dcterms.type = "beslissing op verzoek art. 3 Wob"', up_to=50000, at_a_time=100, callback=plooi_callback)
    # that seems to be a TOOI value, maybe there's a waardelijst somewhere?
    # ...seems to be: https://repository.officiele-overheidspublicaties.nl/waardelijsten/ccw_plooi_documentsoorten_portaal/1/xml/ccw_plooi_documentsoorten_portaal_1.xml

    #Also, take a look at: https://tardis.overheid.nl/waardelijsten.html

    #sru_plooi.search_retrieve_many('dcterms.title any "Wob"', up_to=50000, at_a_time=100, callback=plooi_callback)


if 1:
    #for year in range(2007, 2023):
    for year in range(2022, 2015,-1):
        sru_plooi.search_retrieve_many('dcterms.modified>=%04d-01-01 and dcterms.modified<=%04d-02-01'%(year,year), up_to=50000, at_a_time=100, callback=plooi_callback)
        sru_plooi.search_retrieve_many('dcterms.modified>=%04d-02-01 and dcterms.modified<=%04d-03-01'%(year,year), up_to=50000, at_a_time=100, callback=plooi_callback)
        sru_plooi.search_retrieve_many('dcterms.modified>=%04d-03-01 and dcterms.modified<=%04d-04-01'%(year,year), up_to=50000, at_a_time=100, callback=plooi_callback)
        sru_plooi.search_retrieve_many('dcterms.modified>=%04d-04-01 and dcterms.modified<=%04d-05-01'%(year,year), up_to=50000, at_a_time=100, callback=plooi_callback)
        sru_plooi.search_retrieve_many('dcterms.modified>=%04d-05-01 and dcterms.modified<=%04d-06-01'%(year,year), up_to=50000, at_a_time=100, callback=plooi_callback)
        sru_plooi.search_retrieve_many('dcterms.modified>=%04d-06-01 and dcterms.modified<=%04d-07-01'%(year,year), up_to=50000, at_a_time=100, callback=plooi_callback)
        sru_plooi.search_retrieve_many('dcterms.modified>=%04d-07-01 and dcterms.modified<=%04d-08-01'%(year,year), up_to=50000, at_a_time=100, callback=plooi_callback)
        sru_plooi.search_retrieve_many('dcterms.modified>=%04d-08-01 and dcterms.modified<=%04d-09-01'%(year,year), up_to=50000, at_a_time=100, callback=plooi_callback)
        sru_plooi.search_retrieve_many('dcterms.modified>=%04d-09-01 and dcterms.modified<=%04d-10-01'%(year,year), up_to=50000, at_a_time=100, callback=plooi_callback)
        sru_plooi.search_retrieve_many('dcterms.modified>=%04d-10-01 and dcterms.modified<=%04d-11-01'%(year,year), up_to=50000, at_a_time=100, callback=plooi_callback)
        sru_plooi.search_retrieve_many('dcterms.modified>=%04d-11-01 and dcterms.modified<=%04d-12-01'%(year,year), up_to=50000, at_a_time=100, callback=plooi_callback)
        sru_plooi.search_retrieve_many('dcterms.modified>=%04d-12-01 and dcterms.modified<=%04d-12-31'%(year,year), up_to=50000, at_a_time=100, callback=plooi_callback)

print(unktypes)

XMLSyntaxError: Document is empty, line 1, column 1 (<string>, line 1)

## WGK

In [ ]:
#!/usr/bin/python3

# http://zoekservice.overheid.nl/sru/Search?operation=explain&version=1.2&x-connection=eur

from wetsuite.datacollect.koop_repositories import WetgevingsKalender


def wgk_callback(record):
    #recordSchema   = record.find('recordSchema')      # e.g. <recordSchema>http://standaarden.overheid.nl/sru/</recordSchema>
    #recordPacking  = record.find('recordPacking')     # probably <recordPacking>xml</recordPacking>
    recordData     = record.find('recordData')        # the actual record 
    recordPosition = record.find('recordPosition')    # e.g. <recordPosition>12</recordPosition>
    print( '====== record %s ====='%recordPosition.text )
    payload = recordData[0]
    
    etree.indent_inplace(payload)
    print( etree.tostring( payload, encoding='unicode' ) )

    originalData = payload.find('originalData')
    owmskern     = etree.kvelements_to_dict( originalData.find('meta/owmskern')   )
    owmsmantel   = etree.kvelements_to_dict( originalData.find('meta/owmsmantel') ) 
    #TODO: parse other parts
    
    merged = {}
    merged.update(owmskern)
    merged.update(owmsmantel)

    # TODO: figure out data model
    enrichedData = payload.find('enrichedData')
    for e in enrichedData.findall('locatie'):
        try:
            db.cached_fetch( e.text, verbose=True, given_conn=conn )            
        except Exception as e:
            print('ERROR %s'%e )

    #pprint.pprint( merged )



if 1:
    sru_wgk = WetgevingsKalender()
    pprint.pprint( sru_wgk.explain_parsed() )

    #sru_wgk.search_retrieve_many('dt.title any tolheffing', up_to=50000, at_a_time=500, callback=wgk_callback, verbose=True)

    #sru_wgk.search_retrieve_many('dt.identifier any kst-36137', up_to=50000, at_a_time=500, callback=wgk_callback, verbose=True)

    sru_wgk.search_retrieve_many('w.dossiernummer = 34861', up_to=50000, at_a_time=500, callback=wgk_callback)

    #sru_wgk.search_retrieve_many('dt.created>=2021-01-01', up_to=50000, at_a_time=500, callback=wgk_callback, verbose=True)
    #sru_wgk.search_retrieve_many('dt.modified>=2021-01-01', up_to=50000, at_a_time=500, callback=wgk_callback, verbose=True)
    #sru_wgk.search_retrieve_many('dt.date>=2021-01-01', up_to=50000, at_a_time=500, callback=wgk_callback, verbose=True)
    #sru_wgk.search_retrieve_many('dt.date>=2021-01-01', up_to=50000, at_a_time=500, callback=wgk_callback, verbose=True)






## Samenwerkende catalogi

In [ ]:

dataset = {
    'description':'''
    
    ''', 
    'data':[],
}


ah_tk_sets = collections.defaultdict( list )

print("QUERY")
curs.execute("SELECT url FROM fetched where url like 'https://repository.overheid.nl/frbr/samenwerkendecatalogi/%' -- LIMIT 1000 ")

print("FETCH %d rows"%curs.rowcount)
rows = curs.fetchall()
conn.rollback()


for url, in rows:
    bytestring, _,_ = wetsuite.datacollect.db.cached_fetch( url, dbonly=True, given_conn=conn )
    metadata_tree   = wetsuite.helpers.etree.fromstring(   bytestring )

    tree = wetsuite.helpers.etree.strip_namespace( metadata_tree )

    merged = collections.defaultdict( list )
    for node in [metadata_tree.find('meta/owmskern'), metadata_tree.find('meta/owmsmantel'), metadata_tree.find('meta/scmeta')]:
        for ch in node:
            merged[ch.tag].append( ch.text )

    #pprint.pprint(  merged )
    dataset['data'].append( merged )

with open('samenwerkendecatalogi.json','w', encoding='utf8') as f:
    f.write( json.dumps(dataset) )


## EUR


In [ ]:

# http://zoekservice.overheid.nl/sru/Search?operation=explain&version=1.2&x-connection=eur

from wetsuite.datacollect.koop_repositories import EuropeseRichtlijnen


unkman = collections.defaultdict(int)

def eur_callback(record):
    #recordSchema   = record.find('recordSchema')      # e.g. <recordSchema>http://standaarden.overheid.nl/sru/</recordSchema>
    #recordPacking  = record.find('recordPacking')     # probably <recordPacking>xml</recordPacking>
    recordData     = record.find('recordData')        # the actual record 
    recordPosition = record.find('recordPosition')    # e.g. <recordPosition>12</recordPosition>
    print( '====== record %s ====='%recordPosition.text )
    payload = recordData[0]
    
    wetsuite.helpers.etree.indent_inplace(payload)
    print( wetsuite.helpers.etree.tostring( payload, encoding='unicode' ) )

    originalData = payload.find('originalData')
    owmskern     = wetsuite.helpers.etree.kvelements_to_dict( originalData.find('meta/owmskern')   )
    owmsmantel   = wetsuite.helpers.etree.kvelements_to_dict( originalData.find('meta/owmsmantel') ) 
    #TODO: parse other parts
    
    merged = {}
    merged.update(owmskern)
    merged.update(owmsmantel)

    tried = 0
    # TODO: figure out data model
    enrichedData = payload.find('enrichedData')
    for e in enrichedData.findall('locatie'):
        db.cached_fetch( e.text, verbose=True, given_conn=conn )            

    #pprint.pprint( merged )


if 1:
    sru_eur = EuropeseRichtlijnen()
    #pprint.pprint( sru_eur.explain_parsed() )

    sru_eur.search_retrieve_many('dt.modified >= 1980-01-01', up_to=500, at_a_time=100, callback=eur_callback, verbose=1)
    # This SRU endpoint may be broken?
    
print(unkman)

XMLSyntaxError: Document is empty, line 1, column 1 (<string>, line 1)